<a href="https://colab.research.google.com/github/michalis0/DataMining_and_MachineLearning/blob/master/week6/Text_Analytics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Mining and Machine Learning - Week 6
# Text Analytics

[Text Analytics](https://people.ischool.berkeley.edu/~hearst/text-mining.html) (or text mining) is the process of deriving high-quality information from text. It involves "the discovery by computer of new, previously unknown information, by automatically extracting information from different written resources." Written resources may include websites, books, emails, reviews, and articles.

### Table of Contents
#### 1. Summary
* 1.1 Applications
* 1.2 Tokenization and Stopwords
* 1.3 Stemming and Lemmatization
* 1.4 Text Representation

#### 2. Text Preparation
* 2.1 Install spaCy
* 2.2 Tokenization
* 2.3 Dependency Parsing
* 2.4 Remove Stopwords
* 2.5 Lemmatization
* 2.6 Entity Detection
* 2.7 Exercise
* 2.8 Solution

#### 3. Text Representation
* 3.1 Bag of Words (BOW)
* 3.2 TF-IDF Representation
* 3.3 Exercise
* 3.4 Solution

#### 4. Text Classification: Alexa Reviews
* 4.1 Load and prepare data
* 4.2 Classification of the reviews using logistic regression
* 4.3 How can we improve the accuracy?

## 1. Summary

### 1.1 Applications
There are many applications of text analytics, for example:
* Search for relevant websites or articles using a search engine
* Sentiment Analysis (e.g. classify tweets or film reviews as positive, neutral or negative)
* Chatbots (e.g. Siri, Alexa)
* Project idea: The Impact of Donald Trump’s Tweets on Financial Markets
* Etc.

### 1.2 Tokenization and Stopwords
Tokens are the elementary building blocks (words, numbers, characters) in a document. Tokenization is the process of splitting an input
sequence into tokens. Example: "I love data science" --> "I", "love", "data", "science". Stopwords are common words that appear very frequently (e.g. "is", "and", "you", etc.). It is convenient to remove them as they do not add much to the content of a document and are therefore generally not useful for text analysis or, worse still, make it worse by adding noise.

### 1.3 Lemmatization and Stemming
* Goal: have the same token for different forms of a word (e.g. fishing, fished, fisher, fishers, etc.)
* Lemmatization: Find what is the lemma of a word (e.g. feet -> foot)
* Stemming: one method for lemmatization where rules that remove the ending of a word are applied (e.g. fishing -> fish)


### 1.4 Text Representation
* Goal: transform text such that it can be used for text analysis
* Bag of Words (BOW): works in many case but order is not preserved (solution: n-grams)
* TF-IDF: emphasizes important words

## 2. Text Preparation
In this section, we explain how to prepare a text for analysis. This includes tockeninzing the text, removing stopwords, etc.

### 2.1 Install spaCy
[spaCy](https://spacy.io/) is an open-source natural language processing library for Python. It is designed particularly for production use, and it can help us to build applications that process massive volumes of text efficiently.

We install the library and its English-language model.

In [1]:
# Install and update spaCy
!pip install spacy

# Download the english language model
!python -m spacy download en

     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
      --------------------------------------- 0.3/12.8 MB 5.7 MB/s eta 0:00:03
     - -------------------------------------- 0.6/12.8 MB 5.4 MB/s eta 0:00:03
     - -------------------------------------- 0.6/12.8 MB 5.4 MB/s eta 0:00:03
     - -------------------------------------- 0.6/12.8 MB 5.4 MB/s eta 0:00:03
     - -------------------------------------- 0.6/12.8 MB 5.4 MB/s eta 0:00:03
     - -------------------------------------- 0.6/12.8 MB 5.4 MB/s eta 0:00:03
     -- ------------------------------------- 0.8/12.8 MB 2.5 MB/s eta 0:00:05
     -- ------------------------------------- 0.8/12.8 MB 2.5 MB/s eta 0:00:05
     --- ------------------------------------ 1.0/12.8 MB 2.2 MB/s eta 0:00:06
     --- ------------------------------------ 1.0/12.8 MB 2.2 MB/s eta 0:00:06
     --- ------------------------------------ 1.0/12.8 MB 2.2 MB/s eta 0:00:06
     --- ------------------------------------ 1.0/12.8 MB 2

In [2]:
# Import required packages
import spacy
from spacy import displacy
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import pandas as pd

### 2.2 Tokenization

Tokenization is the process of breaking a text into pieces called tokens. A token simply refers to an individual part of a sentence having some semantic value. SpaCy‘s tokenizer takes input in form of unicode text and outputs a sequence of token objects. In addition, SpaCy automatically breaks your document into tokens when a document is created using the language model.

Let’s take a look at a simple example. Imagine we have the following text, and we would like to tokenize it:

> When learning data science, you shouldn't get discouraged!

> Challenges and setbacks aren't failures, they're just part of the journey. You've got this!

There are a couple of different ways we can appoach this. The first is called __word tokenization__, which means breaking up the text into individual words. This is a critical step for many language processing applications, as they often require inputs in the form of individual words rather than longer strings of text.

In [3]:
# Load English language model
sp = spacy.load('en_core_web_sm')

# Declare text
text = """When learning data science, you shouldn't get discouraged!
Challenges and setbacks aren't failures, they're just part of the journey. You've got this!"""

# spaCy object is used to create a document
my_doc = sp(text)

my_doc

When learning data science, you shouldn't get discouraged!
Challenges and setbacks aren't failures, they're just part of the journey. You've got this!

In [4]:
# This is a spaCy document
type(my_doc)

spacy.tokens.doc.Doc

In [5]:
# Create list of tokens
token_list = []

for token in my_doc:
    token_list.append(token.text)

token_list

['When',
 'learning',
 'data',
 'science',
 ',',
 'you',
 'should',
 "n't",
 'get',
 'discouraged',
 '!',
 '\n',
 'Challenges',
 'and',
 'setbacks',
 'are',
 "n't",
 'failures',
 ',',
 'they',
 "'re",
 'just',
 'part',
 'of',
 'the',
 'journey',
 '.',
 'You',
 "'ve",
 'got',
 'this',
 '!']

As we can see, spaCy produces a list that contains each token as a separate item. Notice that it has recognized that contractions such as _shouldn’t_ actually represent two distinct words, and has thus broken them down into two distinct tokens.

In the example above, we first load the language dictionary. Here we load the english one using `spacy.load('en_core_web_sm')` create an object of this class, "sp", which is used to create documents with linguistic annotations and various language properties. After creating the document, we create a list of tokens.

We can also see the parts-of-speech (POS) of each of these tokens using the `.pos_` attribute, as shown below. POS tagging can be really useful, particularly if you have words or tokens that can have multiple POS tags. For instance, the word "fish" can be used as both a noun and verb, depending upon the context.

In [6]:
# POS
for word in my_doc:
    print(word.text, word.pos_)

When SCONJ
learning VERB
data NOUN
science NOUN
, PUNCT
you PRON
should AUX
n't PART
get AUX
discouraged VERB
! PUNCT

 SPACE
Challenges NOUN
and CCONJ
setbacks NOUN
are AUX
n't PART
failures NOUN
, PUNCT
they PRON
're AUX
just ADV
part NOUN
of ADP
the DET
journey NOUN
. PUNCT
You PRON
've AUX
got VERB
this PRON
! PUNCT


In [7]:
# Another example
doc1 = sp("I like to fish") # verb
doc2 = sp("I eat a fish") # noun

for word in doc1:
  print(word.text, word.pos_)

print("-----------------")

for word in doc2:
  print(word.text, word.pos_)

I PRON
like VERB
to PART
fish VERB
-----------------
I PRON
eat VERB
a DET
fish NOUN



If we want, we can also break the text into sentences rather than words. This is called __sentence tokenization__. When performing sentence tokenization, the tokenizer looks for specific characters that normally fall between sentences, like periods, exclaimation points, and newline characters.

In [8]:
# create list of sentence tokens
sents_list = []

for sent in my_doc.sents:
    sents_list.append(sent.text)

sents_list

["When learning data science, you shouldn't get discouraged!\n",
 "Challenges and setbacks aren't failures, they're just part of the journey.",
 "You've got this!"]

### 2.3 Dependency Parsing
__Depenency parsing__ is a language processing technique that allows to better determine the meaning of a sentence by analyzing how it is constructed to determine how the individual words relate to each other.

Consider, for example, the sentence “Joe throws the ball.” We have two nouns (Joe and ball) and one verb (throws). But we can’t just look at these words individually, or we may end up thinking that the ball throws Joe! To understand the sentence correctly, we need to look at the word order and sentence structure, not just the words.

Below, we have a short sentence. We’ll use a spaCy method called `noun_chunks`, which breaks the input down into nouns and the words describing them, and iterate through each chunk in our source text, identifying the word, its root, its dependency identification, and which chunk it belongs to.

In [9]:
doc = sp(" Joe threw a ball, and President Donald, in pursuit of the ball, hit a wall.") # notice the space at the beginning

for chunk in doc.noun_chunks:
    print(chunk.text, chunk.root.text, chunk.root.dep_,
          chunk.root.head.text)

 Joe Joe nsubj threw
a ball ball dobj threw
President Donald Donald nsubj hit
pursuit pursuit pobj in
the ball ball pobj of
a wall wall dobj hit


In [10]:
# Let's visualize this
displacy.render(doc, style="dep", jupyter= True, options={'distance': 120})

### 2.4 Remove Stopwords
Most text data that we work with is going to contain a lot of words that are not actually useful to us. These words, called stopwords, are useful in human speech, but they do not have much to contribute to data analysis. Removing stopwords helps us eliminate noise and distraction from our text data, and also speeds up the time analysis takes (since there are fewer words to process). This makes text analysis more efficient.

Let’s take a look at the stopwords spaCy includes by default.

In [11]:
# Import stopwords from English language
spacy_stopwords = spacy.lang.en.stop_words.STOP_WORDS

# Print total number of stopwords
print('Number of stopwords: %d' % len(spacy_stopwords))

# Print 20 stopwords
print('20 stopwords: %s' % list(spacy_stopwords)[:20])

Number of stopwords: 326
20 stopwords: ['then', 'while', 'than', 'of', 'could', 'do', 'through', '’d', 'most', 'seems', 'a', 'that', 'although', 'thru', 'never', 'none', "n't", 'get', 'itself', "'m"]


Now that we’ve got our list of stopwords, let’s use it to remove the stopwords from the text string we were working on in the previous section.

In [12]:
# Which words will be removed?
my_doc

When learning data science, you shouldn't get discouraged!
Challenges and setbacks aren't failures, they're just part of the journey. You've got this!

In [13]:
# Declare list for filtered sentence
filtered_sent = []

# Filter stopwords
for word in my_doc:
    if word.is_stop == False:
        filtered_sent.append(word.text)

filtered_sent

['learning',
 'data',
 'science',
 ',',
 'discouraged',
 '!',
 '\n',
 'Challenges',
 'setbacks',
 'failures',
 ',',
 'journey',
 '.',
 'got',
 '!']

In [14]:
# We can also remove the punctuation
filtered_sent2 = []
removed_tokens = []

# Filter stopwords, punctuation and spaces
for word in my_doc:
  if (word.is_stop == True) or (word.is_punct == True) or (word.is_space == True):
    removed_tokens.append(word.text)
  else:
    filtered_sent2.append(word.text)

removed_tokens

['When',
 ',',
 'you',
 'should',
 "n't",
 'get',
 '!',
 '\n',
 'and',
 'are',
 "n't",
 ',',
 'they',
 "'re",
 'just',
 'part',
 'of',
 'the',
 '.',
 'You',
 "'ve",
 'this',
 '!']

In [15]:
filtered_sent2

['learning',
 'data',
 'science',
 'discouraged',
 'Challenges',
 'setbacks',
 'failures',
 'journey',
 'got']

### 2.5 Lemmatization
Lemmatization is a way of dealing with the fact that while words like connect, connection, connecting, connected, etc. aren’t exactly the same, they all have the same essential meaning: connect. The differences in spelling have grammatical functions in spoken language, but for machine processing, those differences can be confusing, so we need a way to change all the words that are forms of the word connect into the word connect itself.

One method for doing this is called __stemming__. Stemming involves simply lopping off easily-identified prefixes and suffixes to produce what’s often the simplest version of a word, the root. Connection, for example, would have the -ion suffix removed and be correctly reduced to connect. This kind of simple stemming is often all that’s needed, but lemmatization—which actually looks at words and their roots (called lemma) as described in the dictionary—is more precise (e.g feet -> foot).

Let's look at this simple example.

In [16]:
# Lemmatization
lem = sp("run runs ran running runner runners")

# Find lemma for each word
for word in lem:
    print(word.text, word.lemma_)

run run
runs run
ran run
running run
runner runner
runners runner


### 2.6 Entity Detection

__Entity detection__, also called entity recognition, is a more advanced form of language processing that identifies important elements like places, people, organizations, and languages within a text. This is really helpful for quickly extracting information from the text, since you can quickly pick out important topics or indentify key sections of it.

Let’s try out some entity detection using a few paragraphs from this [article](https://www.bloomberg.com/features/trump-tweets-market/).

In [17]:
article = sp("""
President Donald Trump gets a lot of attention for using Twitter to attack American trading partners, political foes, and media companies. But he often takes to the platform to celebrate the strength of the world’s largest economy and its publicly-traded companies.

Before U.S. stocks peaked in late January, he drew a direct connection between the increase in market value of American companies and his administration’s pro-growth policies on more than 10 occasions in that month alone.
""")

entities = [(i, i.label_, i.label) for i in article.ents]
entities

[(Donald Trump, 'PERSON', 380),
 (Twitter, 'PERSON', 380),
 (American, 'NORP', 381),
 (U.S., 'GPE', 384),
 (late January, 'DATE', 391),
 (American, 'NORP', 381),
 (more than 10, 'CARDINAL', 397),
 (that month alone, 'DATE', 391)]

The above example how spaCy is able to identify a variety of different entity types, including specific locations (GPE), date-related words (DATE), important numbers (CARDINAL), specific individuals (PERSON), etc.

Using `displaCy` we can also visualize the text, with each identified entity highlighted by a color and labeled. We’ll use `style="ent"` to tell displaCy that we want to visualize entities here.

In [18]:
displacy.render(article, style="ent", jupyter=True)

### 2.7 Exercise
For each word in the sentence below, print its lemma.

In [19]:
sentence = """The happiness of your life depends upon the quality of your thoughts: therefore, guard accordingly, and take care that you entertain no notions unsuitable to virtue and reasonable nature."""

# Print lemma
# [YOUR CODE HERE]

Create two lists, the first one containing the punctuation and the second one the words (tokens).

In [20]:
# [YOUR CODE HERE]

### 2.8 Solution

In [21]:
# Lemma
doc = sp(sentence)
for word in doc:
  print(word.text, word.lemma_)

# punct and tokens
punct = []
tokens = []
for word in doc:
  if word.is_punct == True:
    punct.append(word)
  elif word.is_space == False:
    tokens.append(word)

print(punct)
print(tokens)

The the
happiness happiness
of of
your your
life life
depends depend
upon upon
the the
quality quality
of of
your your
thoughts thought
: :
therefore therefore
, ,
guard guard
accordingly accordingly
, ,
and and
take take
care care
that that
you you
entertain entertain
no no
notions notion
unsuitable unsuitable
to to
virtue virtue
and and
reasonable reasonable
nature nature
. .
[:, ,, ,, .]
[The, happiness, of, your, life, depends, upon, the, quality, of, your, thoughts, therefore, guard, accordingly, and, take, care, that, you, entertain, no, notions, unsuitable, to, virtue, and, reasonable, nature]


## 3. Text Representation
We now show how to transform a text into an usable input for text classification. We use the first sentence of the article from the last section and two other sentences.

In [22]:
# Article as a string, not a spacy object
article = """
President Donald Trump gets a lot of attention for using Twitter to attack American trading partners, political foes, and media companies."""

# Sentences
s1 = """Donald Trump is a great friend, and he has four or five Picassos on his plane. And that's where I would look at them.""" # from Shaquille O'Neal
s2 = """Donald Trump is a phony, a fraud. His promises are as worthless as a degree from Trump University.""" # from Mitt Romney

# List of sentences
texts = [article, s1, s2]

### 3.1 Bag of Words (BOW)
We use the [CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) class of scikit learn.

In [23]:
# Using default tokenizer 
count = CountVectorizer(ngram_range=(1,2), stop_words="english")
bow = count.fit_transform(texts)

# Show feature matrix
bow.toarray()

array([[1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0,
        1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1,
        0, 0, 0, 1, 1, 0, 1, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
        1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 2, 0,
        0, 1, 1, 0, 0, 1, 0, 0, 1, 1]], dtype=int64)

In [24]:
# Get feature names
feature_names = count.get_feature_names_out()

# View feature names
feature_names

array(['american', 'american trading', 'attack', 'attack american',
       'attention', 'attention using', 'companies', 'degree',
       'degree trump', 'donald', 'donald trump', 'foes', 'foes media',
       'fraud', 'fraud promises', 'friend', 'friend picassos', 'gets',
       'gets lot', 'great', 'great friend', 'look', 'lot',
       'lot attention', 'media', 'media companies', 'partners',
       'partners political', 'phony', 'phony fraud', 'picassos',
       'picassos plane', 'plane', 'plane look', 'political',
       'political foes', 'president', 'president donald', 'promises',
       'promises worthless', 'trading', 'trading partners', 'trump',
       'trump gets', 'trump great', 'trump phony', 'trump university',
       'twitter', 'twitter attack', 'university', 'using',
       'using twitter', 'worthless', 'worthless degree'], dtype=object)

In [25]:
# Show as a dataframe
pd.DataFrame(
    bow.todense(), 
    columns=feature_names
    )

,american,american trading,attack,attack american,attention,attention using,companies,degree,degree trump,donald,...,trump great,trump phony,trump university,twitter,twitter attack,university,using,using twitter,worthless,worthless degree
0,1,1,1,1,1,1,1,0,0,1,...,0,0,0,1,1,0,1,1,0,0
1,0,0,0,0,0,0,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,1,1,1,...,0,1,1,0,0,1,0,0,1,1


### 3.2 TF-IDF Representation


Recall that:

- term frequency tf = count(word, document) / len(document) 
- term frequency idf = log( len(collection) / count(document_containing_term, collection) )
- tf-idf = tf * idf 

It is important to mention that the IDF value for a word remains the same throughout all the documents as it depends upon the total number of documents. On the other hand, TF values of a word differ from document to document.


In [26]:
# Using default tokenizer in TfidfVectorizer
tfidf = TfidfVectorizer(ngram_range=(1, 1), stop_words="english")
features = tfidf.fit_transform(texts)
pd.DataFrame(
    features.todense(),
    columns=tfidf.get_feature_names_out()
)

,american,attack,attention,companies,degree,donald,foes,fraud,friend,gets,...,plane,political,president,promises,trading,trump,twitter,university,using,worthless
0,0.260841,0.260841,0.260841,0.260841,0.000000,0.154057,0.260841,0.000000,0.00000,0.260841,...,0.00000,0.260841,0.260841,0.000000,0.260841,0.154057,0.260841,0.000000,0.260841,0.000000
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.247433,0.000000,0.000000,0.41894,0.000000,...,0.41894,0.000000,0.000000,0.000000,0.000000,0.247433,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.359347,0.212236,0.000000,0.359347,0.00000,0.000000,...,0.00000,0.000000,0.000000,0.359347,0.000000,0.424472,0.000000,0.359347,0.000000,0.359347


In [27]:
texts

['\nPresident Donald Trump gets a lot of attention for using Twitter to attack American trading partners, political foes, and media companies.',
 "Donald Trump is a great friend, and he has four or five Picassos on his plane. And that's where I would look at them.",
 'Donald Trump is a phony, a fraud. His promises are as worthless as a degree from Trump University.']

### 3.3 Exercise
Create a TF-IDF Representation of the three above sentences using bigrams and the following stopwords: ["and", "a", "is"].

In [28]:
# YOUR CODE HERE

### 3.4 Solution

In [29]:
tfidf = TfidfVectorizer(ngram_range=(2, 2), stop_words=["and", "a", "is"])
features = tfidf.fit_transform(texts)
pd.DataFrame(
    features.todense(),
    columns=tfidf.get_feature_names_out()
)

,american trading,are as,as degree,as worthless,at them,attack american,attention for,degree from,donald trump,five picassos,...,trading partners,trump gets,trump great,trump phony,trump university,twitter to,using twitter,where would,worthless as,would look
0,0.240085,0.000000,0.000000,0.000000,0.000000,0.240085,0.240085,0.000000,0.141798,0.000000,...,0.240085,0.240085,0.000000,0.000000,0.000000,0.240085,0.240085,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.000000,0.240085,0.000000,0.000000,0.000000,0.141798,0.240085,...,0.000000,0.000000,0.240085,0.000000,0.000000,0.000000,0.000000,0.240085,0.000000,0.240085
2,0.000000,0.284569,0.284569,0.284569,0.000000,0.000000,0.000000,0.284569,0.168071,0.000000,...,0.000000,0.000000,0.000000,0.284569,0.284569,0.000000,0.000000,0.000000,0.284569,0.000000


## 4. Text Classification: Alexa reviews

We are going to use a real-world data set: [Amazon Alexa product reviews](https://www.kaggle.com/bittlingmayer/amazonreviews).

This data set comes as a tab-separated file (.tsv). It has has five columns: `rating`, `date`, `variation`, `verified_reviews`, `feedback`.

`rating` denotes the rating each user gave Alexa (out of 5). `date` indicates the date of the review, and `variation` describes which model the user reviewed. `verified_reviews` contains the text of each review, and `feedback` contains a sentiment label, with 1 denoting positive sentiment (the user liked it) and 0 denoting negative sentiment (the user didn’t).

We are going develop a classification model that looks at the review text and predicts whether a review is positive or negative. Since this data set already includes whether a review is positive or negative in the `feedback` column, we can use those answers to train and test our model. Our goal here is to produce an accurate model that we could then use to process new user reviews and quickly determine whether they were positive or negative.

### 4.1 Load and prepare data

In [30]:
# Import additional packages
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
import string
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

In [31]:
# Load data
url = "https://raw.githubusercontent.com/LKunz/Projets/refs/heads/main/Data%20Science/data/amazon_alexa.tsv"
df = pd.read_csv(url, delimiter="\t")
df.sample(10)

,rating,date,variation,verified_reviews,feedback
692,5,17-May-18,Black,Amazingly fun. My daughter and I just asked qu...,1
3102,5,30-Jul-18,White Dot,"Works great, love the fact you can play the sa...",1
2096,5,29-Jun-18,Black Plus,We have a great time using this as a family. E...,1
2263,5,30-Jul-18,Configuration: Fire TV Stick,Worked as advertised !,1
1988,4,22-Jul-18,Black Plus,I love it but does not play well with my Amazo...,1
1164,4,29-Jul-18,White Spot,so far just ok,1
1930,5,26-Jul-18,White Plus,"Great device, my grandkids had a ball setting ...",1
2995,4,30-Jul-18,Black Dot,Small speaker tin sound. Great with good Bluet...,1
2902,5,30-Jul-18,Black Dot,Excellent. Bought another one yesterday!,1
1130,5,29-Jul-18,Black Spot,Hands free control,1


In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3150 entries, 0 to 3149
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   rating            3150 non-null   int64 
 1   date              3150 non-null   object
 2   variation         3150 non-null   object
 3   verified_reviews  3150 non-null   object
 4   feedback          3150 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 123.2+ KB


In [33]:
# Change date to datetime
df["date"] = pd.to_datetime(df["date"])

In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3150 entries, 0 to 3149
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   rating            3150 non-null   int64         
 1   date              3150 non-null   datetime64[ns]
 2   variation         3150 non-null   object        
 3   verified_reviews  3150 non-null   object        
 4   feedback          3150 non-null   int64         
dtypes: datetime64[ns](1), int64(2), object(2)
memory usage: 123.2+ KB


In [35]:
# Base rate: the data-set is unbalanced!
df.feedback.value_counts()

1    2893
0     257
Name: feedback, dtype: int64

In [36]:
round(df.feedback.value_counts()[1] / len(df), 4)

0.9184

###### Tokening the Data With spaCy

We create a `spacy_tokenizer()` function that accepts a sentence as input and processes the sentence into tokens, performing lemmatization, lowercasing, and removing stopwords.

__A note from spacy documentation__: spaCy adds a special case for pronouns: all pronouns are lemmatized to the special token `-PRON-`. Unlike verbs and common nouns, there’s no clear base form of a personal pronoun. Should the lemma of “me” be “I”, or should we normalize person as well, giving “it” — or maybe “he”? spaCy’s solution is to introduce a novel symbol, `-PRON-`, which is used as the lemma for all personal pronouns.

In [37]:
# Create a list of punctuation marks
punctuations = string.punctuation

punctuations

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [38]:
# Create a list of stopwords
stop_words = spacy.lang.en.stop_words.STOP_WORDS

list(stop_words)[:10]

['then',
 'while',
 'than',
 'of',
 'could',
 'do',
 'through',
 '’d',
 'most',
 'seems']

In [39]:
# Load English language model
sp = spacy.load('en_core_web_sm')

# Create tokenizer function
def spacy_tokenizer(sentence):
    # Create token object, which is used to create documents with linguistic annotations.
    mytokens = sp(sentence)

    # Lemmatize each token and convert each token into lowercase
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]

    # Remove stop words and punctuation
    mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]

    # Return preprocessed list of tokens
    return mytokens

# Example
review = df["verified_reviews"].sample()
review.values[0]

'We were pleasantly surprised at the sound quality and the many features. How nice to easily hear a particular artist or hear our books just by a simple command. Looking forward to exploring other options with this device.'

In [40]:
spacy_tokenizer(review.values[0])

['pleasantly',
 'surprised',
 'sound',
 'quality',
 'feature',
 'nice',
 'easily',
 'hear',
 'particular',
 'artist',
 'hear',
 'book',
 'simple',
 'command',
 'look',
 'forward',
 'explore',
 'option',
 'device']

#### Vectorization Feature Engineering (TF-IDF)

We use the TF-IDF (Term Frequency-Inverse Document Frequency) to vectorize the documents. This is a way of representing how important a particular term is in the context of a given document, based on how many times the term appears and how many other documents that same term appears in. The higher the TF-IDF, the more important that term is to that document.

In [41]:
tfidf_vector = TfidfVectorizer(tokenizer=spacy_tokenizer) # we use the above defined tokenizer

### 4.2 Classification of the reviews using logistic regression

In [42]:
# Select features
X = df['verified_reviews'] # the features we want to analyze
ylabels = df['feedback'] # the labels, or answers, we want to test against

# Train test split
X_train, X_test, y_train, y_test = train_test_split(X, ylabels, test_size=0.2, random_state=1234)

X_train

2088    I really like this item. I already had the ech...
287     Technology in such a small device. The price w...
516           Pleased with the order just what I wanted!!
832                                               Love it
890                                   Works as advertised
                              ...                        
664     Great for mom its inside an owlhead in her jac...
3125    This product is easy to use and very entertain...
1318    You're going to have to shell out an extra $20...
723     Fun item to play with and get used to using.  ...
2863                                        Great product
Name: verified_reviews, Length: 2520, dtype: object

In [43]:
y_train

2088    1
287     1
516     1
832     1
890     1
       ..
664     1
3125    1
1318    1
723     1
2863    1
Name: feedback, Length: 2520, dtype: int64

In [44]:
# Define classifier
classifier = LogisticRegression()

# Create pipeline
pipe = Pipeline([('vectorizer', tfidf_vector),
                 ('classifier', classifier)])

# Fit model on training set
pipe.fit(X_train, y_train)

C:\Users\lucku\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('vectorizer',
                 TfidfVectorizer(tokenizer=<function spacy_tokenizer at 0x000001CDE0D208B0>)),
                ('classifier', LogisticRegression())])

In [45]:
# Evaluate the model
def evaluate(true, pred):
    precision = precision_score(true, pred)
    recall = recall_score(true, pred)
    f1 = f1_score(true, pred)
    print(f"CONFUSION MATRIX:\n{confusion_matrix(true, pred)}")
    print(f"ACCURACY SCORE:\n{accuracy_score(true, pred):.4f}")
    print(f"CLASSIFICATION REPORT:\n\tPrecision: {precision:.4f}\n\tRecall: {recall:.4f}\n\tF1_Score: {f1:.4f}")

In [46]:
# Predictions
y_pred = pipe.predict(X_test)

# Evaluation - test set
evaluate(y_test, y_pred)

CONFUSION MATRIX:
[[  2  49]
 [  0 579]]
ACCURACY SCORE:
0.9222
CLASSIFICATION REPORT:
	Precision: 0.9220
	Recall: 1.0000
	F1_Score: 0.9594


For the test set, the model correctly identifies a sentiment 92.22% of the time. This is only slightly better than the base rate (91.84%). The difference lies in the two cases that the model correctly classifies as negative. Therefore, the model does not work very well. This may be due to the fact that we have an unbalanced sample with too much positive reviews. Maybe the model cannot learn how to classify negative reviews well since there are too few examples of them.

A recall of 1 means that if a sentiment is positive, it will be classififed as positive.

We observe approximately the same on the training set, as shown below.

In [47]:
# Evaluate on training set
evaluate(y_train, pipe.predict(X_train))

CONFUSION MATRIX:
[[   6  200]
 [   0 2314]]
ACCURACY SCORE:
0.9206
CLASSIFICATION REPORT:
	Precision: 0.9204
	Recall: 1.0000
	F1_Score: 0.9586


In [48]:
# Which reviews are classified as negative in test set?
X_test.loc[pipe.predict(X_test) == 0]

491                            I've already returned it.
381    It worked for a month or so then it stopped. I...
Name: verified_reviews, dtype: object

In [49]:
# Which reviews are classified as negaive in training set?
X_train.loc[pipe.predict(X_train) == 0].values

array(["Can't turn of &#34;things to try&#34; on the screen scroll. This  is a big promotion device for Amazon.  Packing it up and sending it back.  They really want to get in our heads.  Enough already.",
       "This worked well for about 6 months but then stopped connecting to the internet (wifi) and none of the trouble shooting efforts worked.  Amazon customer service couldn't help further and told me warranty was only 3 months, which was very disappointing, so they weren't willing to replace.",
       'product stopped working after return time ran out',
       'Item has never worked. Out of box it is broken. Spent several days trying to get it working and running same &#34;fixes&#34; from Amazon. The only thing accomplished is I will never order another refurbished device.',
       'Terrible. Stopped working after one day.',
       "This worked well for about 6 months but then stopped connecting to the internet (wifi) and none of the trouble shooting efforts worked.  Amazon custom

In [50]:
# Prediction for new reviews
example_review_1 = "I really love the product. It is very helpful. I use it everyday" # positive
example_review_2 = "It stopped working, I want to return it" # negative
example_review_3 = "I don't like it, it is bad" # negative

examples = pd.Series([example_review_1, example_review_2, example_review_3])
examples

0    I really love the product. It is very helpful....
1              It stopped working, I want to return it
2                           I don't like it, it is bad
dtype: object

In [51]:
pipe.predict(examples)

array([1, 0, 1], dtype=int64)

### 4.3 How can we improve the accuracy?
In order to improve the prediction, we can try to:
* Resample our data (i.e. add negative examples for better training)
* Tun the hyperparameters of the model
* Improve text preparation
* Use another classififer

We illustrate how to improve text preparation, resampling, and the use of another classifier below.

#### 4.3.1 Improve text preparation
The purpose here is to optimize the parameters of the [TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) class.

In [52]:
# Create list of configs
def configs():

    models = list()
    
    # Define config lists
    ngram_range = [(1,1), (1,2), (1, 3), (2, 2), (2, 3), (3, 3)]
    min_df = [1]
    max_df = [1.0]
    analyzer=['word', 'char']
    
    # Create config instances
    for n in ngram_range:
        for i in min_df:
            for j in max_df:
              for a in analyzer:
                    cfg = [n, i, j, a]
                    models.append(cfg)
    return models

configs = configs()
configs[:10]

[[(1, 1), 1, 1.0, 'word'],
 [(1, 1), 1, 1.0, 'char'],
 [(1, 2), 1, 1.0, 'word'],
 [(1, 2), 1, 1.0, 'char'],
 [(1, 3), 1, 1.0, 'word'],
 [(1, 3), 1, 1.0, 'char'],
 [(2, 2), 1, 1.0, 'word'],
 [(2, 2), 1, 1.0, 'char'],
 [(2, 3), 1, 1.0, 'word'],
 [(2, 3), 1, 1.0, 'char']]

In [53]:
# Define list for result
result = []

for config in configs:

    # Redefine vectorizer
    tfidf_vector = TfidfVectorizer(tokenizer=spacy_tokenizer, 
                                   ngram_range=config[0],
                                   min_df=config[1], max_df=config[2], analyzer=config[3])

    # Define classifier
    classifier = LogisticRegression()

    # Create pipeline
    pipe = Pipeline([('vectorizer', tfidf_vector),
                 ('classifier', classifier)])

    # Fit model on training set
    pipe.fit(X_train, y_train)

    # Predictions
    y_pred = pipe.predict(X_test)

    # Print accuracy on test set
    print("CONFIG: ", config)
    evaluate(y_test, y_pred)
    print("-----------------------")

    # Append to result
    result.append([config, accuracy_score(y_test, y_pred)])

C:\Users\lucku\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


CONFIG:  [(1, 1), 1, 1.0, 'word']
CONFUSION MATRIX:
[[  2  49]
 [  0 579]]
ACCURACY SCORE:
0.9222
CLASSIFICATION REPORT:
	Precision: 0.9220
	Recall: 1.0000
	F1_Score: 0.9594
-----------------------
CONFIG:  [(1, 1), 1, 1.0, 'char']
CONFUSION MATRIX:
[[  0  51]
 [  0 579]]
ACCURACY SCORE:
0.9190
CLASSIFICATION REPORT:
	Precision: 0.9190
	Recall: 1.0000
	F1_Score: 0.9578
-----------------------


C:\Users\lucku\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
C:\Users\lucku\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:563: UserWarning: The parameter 'tokenizer' will not be used since 'analyzer' != 'word'
  warnings.warn(
C:\Users\lucku\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


CONFIG:  [(1, 2), 1, 1.0, 'word']
CONFUSION MATRIX:
[[  0  51]
 [  0 579]]
ACCURACY SCORE:
0.9190
CLASSIFICATION REPORT:
	Precision: 0.9190
	Recall: 1.0000
	F1_Score: 0.9578
-----------------------


C:\Users\lucku\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
C:\Users\lucku\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:563: UserWarning: The parameter 'tokenizer' will not be used since 'analyzer' != 'word'
  warnings.warn(


...

CONFIG:  [(2, 3), 1, 1.0, 'char']
CONFUSION MATRIX:
[[  1  50]
 [  0 579]]
ACCURACY SCORE:
0.9206
CLASSIFICATION REPORT:
	Precision: 0.9205
	Recall: 1.0000
	F1_Score: 0.9586
-----------------------


C:\Users\lucku\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


CONFIG:  [(3, 3), 1, 1.0, 'word']
CONFUSION MATRIX:
[[  0  51]
 [  0 579]]
ACCURACY SCORE:
0.9190
CLASSIFICATION REPORT:
	Precision: 0.9190
	Recall: 1.0000
	F1_Score: 0.9578
-----------------------


C:\Users\lucku\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
C:\Users\lucku\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:563: UserWarning: The parameter 'tokenizer' will not be used since 'analyzer' != 'word'
  warnings.warn(


CONFIG:  [(3, 3), 1, 1.0, 'char']
CONFUSION MATRIX:
[[  0  51]
 [  0 579]]
ACCURACY SCORE:
0.9190
CLASSIFICATION REPORT:
	Precision: 0.9190
	Recall: 1.0000
	F1_Score: 0.9578
-----------------------


Our tries do not work, we have to try further or use something else to improve prediction.

#### 4.3.2 Resampling

In [54]:
# Create balanced dataframe - base rate = 0.5
df_new = pd.concat([df[df["feedback"] == 1].sample(len(df[df["feedback"] == 0])), df[df["feedback"] == 0]], axis=0).reset_index()
df_new

,index,rating,date,variation,verified_reviews,feedback
0,3012,5,2018-07-30,White Dot,I am spoiled by one downstairs...so got this o...,1
1,2920,5,2018-07-30,Black Dot,Cool little speaker..I love her alarms and con...,1
2,1413,5,2018-07-30,White Show,I like the echo show its the best! big screen ...,1
3,1173,5,2018-07-28,Black Spot,Love these!,1
4,1808,5,2018-07-29,White Plus,,1
...,...,...,...,...,...,...
509,3047,1,2018-07-30,Black Dot,Echo Dot responds to us when we aren't even ta...,0
510,3048,1,2018-07-30,White Dot,NOT CONNECTED TO MY PHONE PLAYLIST :(,0
511,3067,2,2018-07-30,Black Dot,The only negative we have on this product is t...,0
512,3091,1,2018-07-30,Black Dot,I didn’t order it,0


In [55]:
# Select features
X = df_new['verified_reviews'] # the features we want to analyze
ylabels = df_new['feedback'] # the labels, or answers, we want to test against

# Train test split
X_train_b, X_test_b, y_train_b, y_test_b = train_test_split(X, ylabels, test_size=0.2, random_state=1234)

# Define classifier
classifier = LogisticRegression()

# Create pipeline
pipe = Pipeline([('vectorizer', tfidf_vector),
                 ('classifier', classifier)])

# Fit model on training set
pipe.fit(X_train_b, y_train_b)

# Predictions
y_pred_b = pipe.predict(X_test_b)

# Evaluation - test set
evaluate(y_test_b, y_pred_b)

CONFUSION MATRIX:
[[47  9]
 [10 37]]
ACCURACY SCORE:
0.8155
CLASSIFICATION REPORT:
	Precision: 0.8043
	Recall: 0.7872
	F1_Score: 0.7957


C:\Users\lucku\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
C:\Users\lucku\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:563: UserWarning: The parameter 'tokenizer' will not be used since 'analyzer' != 'word'
  warnings.warn(


#### 4.3.3 Use another classifier

In [56]:
# Use random forest
from sklearn.ensemble import RandomForestClassifier

# Define vectorizer
tfidf_vector = TfidfVectorizer(tokenizer=spacy_tokenizer) # we use the above defined tokenizer

# Define classifier
classifier = RandomForestClassifier()

# Create pipeline
pipe = Pipeline([('vectorizer', tfidf_vector),
                 ('classifier', classifier)])

# Fit model on training set
pipe.fit(X_train, y_train)

# Predictions
y_pred = pipe.predict(X_test)

# Evaluation - test set
evaluate(y_test, y_pred)

# Evaluation - training set
evaluate(y_train, pipe.predict(X_train))

C:\Users\lucku\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


CONFUSION MATRIX:
[[ 10  41]
 [  0 579]]
ACCURACY SCORE:
0.9349
CLASSIFICATION REPORT:
	Precision: 0.9339
	Recall: 1.0000
	F1_Score: 0.9658
CONFUSION MATRIX:
[[ 189   17]
 [   0 2314]]
ACCURACY SCORE:
0.9933
CLASSIFICATION REPORT:
	Precision: 0.9927
	Recall: 1.0000
	F1_Score: 0.9963


Of course, combining the three above-mentioned techniques should give the best result.

#### BONUS

In [57]:
# BONUS: predict the rating
df.sample(5)

,rating,date,variation,verified_reviews,feedback
2666,5,2018-07-30,Black Dot,"We are still in the discovery phase, but so fa...",1
2007,5,2018-07-20,Black Plus,Absolutely love it.. only thing that would ma...,1
1142,4,2018-07-29,Black Spot,"Works decent, wish it was able to answer more ...",1
1511,5,2018-07-30,White Show,Upgraded from an original Echo on Prime Day an...,1
1828,4,2018-07-28,Black Plus,I love everything about it. I love that the ec...,1


In [58]:
df.rating.value_counts()

5    2286
4     455
1     161
3     152
2      96
Name: rating, dtype: int64

In [59]:
# Base rate
round(df.rating.value_counts()[5] / len(df), 4)

0.7257

In [60]:
# Select features
X = df['verified_reviews'] # the features we want to analyze
y = df['rating'] # the labels, or answers, we want to test against

# Train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

# Define classifier
classifier = LogisticRegression()

# Create pipeline
pipe = Pipeline([('vectorizer', tfidf_vector),
                 ('classifier', classifier)])

# Generate Model on training set
pipe.fit(X_train, y_train)

# Predictions
y_pred = pipe.predict(X_test)

# Evaluation - test set
print(f"CONFUSION MATRIX:\n{confusion_matrix(y_test, y_pred)}")
print(f"ACCURACY SCORE:\n{accuracy_score(y_test, y_pred):.4f}")

C:\Users\lucku\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


CONFUSION MATRIX:
[[  2   0   0   4  31]
 [  1   1   0   2  10]
 [  0   0   0   3  32]
 [  0   0   0   8  84]
 [  0   0   0   5 447]]
ACCURACY SCORE:
0.7270


In [61]:
# BONUS 2: use random forest

# Define classifier
classifier = RandomForestClassifier(n_estimators=50)

# Create pipeline
pipe = Pipeline([('vectorizer', tfidf_vector),
                 ('classifier', classifier)])

# Generate Model on training set
pipe.fit(X_train, y_train)

# Predictions
y_pred = pipe.predict(X_test)

# Evaluation - test set
print(f"CONFUSION MATRIX:\n{confusion_matrix(y_test, y_pred)}")
print(f"ACCURACY SCORE:\n{accuracy_score(y_test, y_pred):.4f}")

C:\Users\lucku\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


CONFUSION MATRIX:
[[  7   0   0   3  27]
 [  0   4   0   3   7]
 [  0   0  12   4  19]
 [  0   0   0  34  58]
 [  0   0   1   1 450]]
ACCURACY SCORE:
0.8048
